# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [ ]:
import json
from knowrob import *
    
InitKnowRob()

### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [ ]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"},
            {"alias": "dfl", "uri": "http://www.ease-crc.org/ont/SOMA_DFL.owl"}

		]
	},
	"data-sources": [
		{"path": "owl/USD.owl", "format": "rdf-xml"},
		{"path": "owl/house_4.owl", "format": "rdf-xml"}
	],
	"data-backends": [
    {
      "type": "Redland",
      "name": "redland",
      "read-only": False
    }
	],  
    "reasoner": [
    {
      "type": "SOMADFLReasoner",
      "name": "DummyReasoner",
      "module": "reasoner/dfl_reasoner.py",
      "data-backend": "redland"
    }
    ]
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

### Submitting a Query

Now we want to run a query on the loaded semantic map:

In [ ]:
phi1 = QueryParser.parse("dfl:isInstanceOf(?x, 'http://ease-crc.org/ont/SOMA_DFL.owl#breakfast_food.n.wn.food')")
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

if isinstance(nextResult1, AnswerYes):
    for substitution in nextResult1.substitution():
        variable = substitution[1]
        term = substitution[2]
        print(str(variable) + " : " + str(term))

CQ1 Which objects do I need for breakfast/
CQ2 Which objects contain something to drink?
CQ3 Where do we expect an item to be?
CQ4 What can I grasp on an object to open it?
CQ5 Where should I put the utensils for breakfast?

In [ ]:
phi1 = QueryParser.parse("'http://knowrob.org/kb/lpn#jealous'(?s, ?y)")
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

if isinstance(nextResult1, AnswerNo):
    print("Answer is no")